# টেক্সট শ্রেণীবিন্যাস কাজ

যেমনটি আমরা উল্লেখ করেছি, আমরা **AG_NEWS** ডেটাসেটের উপর ভিত্তি করে একটি সহজ টেক্সট শ্রেণীবিন্যাস কাজের উপর মনোযোগ দেব। এই কাজটি হলো সংবাদ শিরোনামগুলোকে চারটি বিভাগে শ্রেণীবদ্ধ করা: বিশ্ব, ক্রীড়া, ব্যবসা এবং বিজ্ঞান/প্রযুক্তি।

## ডেটাসেট

এই ডেটাসেটটি [`torchtext`](https://github.com/pytorch/text) মডিউলে অন্তর্ভুক্ত, তাই আমরা এটি সহজেই অ্যাক্সেস করতে পারি।


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

এখানে, `train_dataset` এবং `test_dataset` এমন সংগ্রহসমূহ ধারণ করে যা যথাক্রমে শ্রেণীর সংখ্যা (লেবেল) এবং টেক্সটের জোড়া প্রদান করে, উদাহরণস্বরূপ:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

তাহলে, চলুন আমাদের ডেটাসেট থেকে প্রথম ১০টি নতুন শিরোনাম প্রিন্ট করি:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

কারণ ডেটাসেটগুলি ইটারেটর, যদি আমরা ডেটা একাধিকবার ব্যবহার করতে চাই তবে আমাদের এটি তালিকায় রূপান্তর করতে হবে:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## টোকেনাইজেশন

এখন আমাদের টেক্সটকে **সংখ্যায়** রূপান্তর করতে হবে যা টেনসর হিসেবে উপস্থাপন করা যায়। যদি আমরা শব্দ-স্তরের উপস্থাপনা চাই, তাহলে আমাদের দুটি কাজ করতে হবে:
* **টোকেনাইজার** ব্যবহার করে টেক্সটকে **টোকেন** এ বিভক্ত করা
* সেই টোকেনগুলোর একটি **ভোকাবুলারি** তৈরি করা।


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

শব্দভাণ্ডার ব্যবহার করে, আমরা সহজেই আমাদের টোকেনাইজড স্ট্রিংকে সংখ্যার একটি সেটে এনকোড করতে পারি:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## শব্দের ব্যাগ (Bag of Words) টেক্সট উপস্থাপনা

কারণ শব্দগুলো অর্থ প্রকাশ করে, কখনও কখনও আমরা শুধুমাত্র পৃথক শব্দগুলো দেখে একটি টেক্সটের অর্থ বুঝতে পারি, বাক্যে তাদের ক্রম না দেখেও। উদাহরণস্বরূপ, সংবাদ শ্রেণীবিভাগ করার সময়, *আবহাওয়া*, *তুষারপাত* এর মতো শব্দগুলো সম্ভবত *আবহাওয়ার পূর্বাভাস* নির্দেশ করবে, যেখানে *শেয়ারবাজার*, *ডলার* এর মতো শব্দগুলো *আর্থিক সংবাদ* এর দিকে ইঙ্গিত করবে।

**শব্দের ব্যাগ** (BoW) ভেক্টর উপস্থাপনাটি সবচেয়ে বেশি ব্যবহৃত ঐতিহ্যবাহী ভেক্টর উপস্থাপনাগুলোর একটি। প্রতিটি শব্দ একটি ভেক্টর সূচকের সাথে যুক্ত থাকে, এবং ভেক্টরের উপাদানটি একটি নির্দিষ্ট ডকুমেন্টে সেই শব্দটির উপস্থিতির সংখ্যা ধারণ করে।

![একটি শব্দের ব্যাগ ভেক্টর উপস্থাপনাটি মেমোরিতে কীভাবে উপস্থাপিত হয় তা দেখানো একটি চিত্র।](../../../../../translated_images/bn/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: BoW-কে আপনি টেক্সটের পৃথক শব্দগুলোর জন্য এক-হট-এনকোডেড ভেক্টরগুলোর যোগফল হিসেবেও ভাবতে পারেন।

নিচে Scikit Learn পাইথন লাইব্রেরি ব্যবহার করে কীভাবে একটি শব্দের ব্যাগ উপস্থাপনা তৈরি করা যায় তার একটি উদাহরণ দেওয়া হলো:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

AG_NEWS ডেটাসেটের ভেক্টর উপস্থাপন থেকে ব্যাগ-অফ-ওয়ার্ডস ভেক্টর গণনা করতে, আমরা নিম্নলিখিত ফাংশনটি ব্যবহার করতে পারি:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **নোট:** এখানে আমরা গ্লোবাল `vocab_size` ভেরিয়েবল ব্যবহার করছি শব্দভাণ্ডারের ডিফল্ট আকার নির্ধারণ করতে। যেহেতু প্রায়শই শব্দভাণ্ডারের আকার বেশ বড় হয়, আমরা সবচেয়ে বেশি ব্যবহৃত শব্দগুলোর উপর ভিত্তি করে শব্দভাণ্ডারের আকার সীমিত করতে পারি। `vocab_size` এর মান কমিয়ে কোডটি চালিয়ে দেখুন এটি কীভাবে সঠিকতাকে প্রভাবিত করে। আপনি কিছুটা সঠিকতার হ্রাস আশা করতে পারেন, তবে তা নাটকীয় নয়, বরং উচ্চতর কার্যক্ষমতার বিনিময়ে।


## BoW শ্রেণীবিন্যাসকারী প্রশিক্ষণ

এখন যেহেতু আমরা শিখেছি কীভাবে আমাদের টেক্সটের Bag-of-Words উপস্থাপনা তৈরি করতে হয়, চলুন এর উপর একটি শ্রেণীবিন্যাসকারী প্রশিক্ষণ দিই। প্রথমে, আমাদের ডেটাসেটকে এমনভাবে রূপান্তর করতে হবে যাতে সমস্ত পজিশনাল ভেক্টর উপস্থাপনাগুলি Bag-of-Words উপস্থাপনায় রূপান্তরিত হয়। এটি অর্জন করা সম্ভব `bowify` ফাংশনকে স্ট্যান্ডার্ড টর্চ `DataLoader`-এর `collate_fn` প্যারামিটার হিসেবে পাস করার মাধ্যমে:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

এখন চলুন একটি সাধারণ শ্রেণীবিন্যাসকারী নিউরাল নেটওয়ার্ক সংজ্ঞায়িত করি যা একটি লিনিয়ার স্তর ধারণ করে। ইনপুট ভেক্টরের আকার `vocab_size` এর সমান, এবং আউটপুট আকার শ্রেণীর সংখ্যা (৪) এর সাথে সঙ্গতিপূর্ণ। যেহেতু আমরা শ্রেণীবিন্যাস কাজ সমাধান করছি, চূড়ান্ত সক্রিয়করণ ফাংশন হলো `LogSoftmax()`।


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

এখন আমরা স্ট্যান্ডার্ড PyTorch প্রশিক্ষণ লুপ সংজ্ঞায়িত করব। যেহেতু আমাদের ডেটাসেটটি বেশ বড়, আমাদের শিক্ষার উদ্দেশ্যে আমরা শুধুমাত্র এক এপোকের জন্য প্রশিক্ষণ দেব, এবং কখনও কখনও এক এপোকের চেয়েও কম (প্রশিক্ষণ সীমিত করতে `epoch_size` প্যারামিটার নির্ধারণ করা হয়)। আমরা প্রশিক্ষণের সময় সংগৃহীত প্রশিক্ষণ সঠিকতা রিপোর্ট করব; রিপোর্ট করার ফ্রিকোয়েন্সি `report_freq` প্যারামিটার ব্যবহার করে নির্ধারণ করা হয়।


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## বাইগ্রাম, ট্রাইগ্রাম এবং এন-গ্রাম

ব্যাগ অফ ওয়ার্ডস পদ্ধতির একটি সীমাবদ্ধতা হলো কিছু শব্দ বহু শব্দের অভিব্যক্তির অংশ হতে পারে। উদাহরণস্বরূপ, 'হট ডগ' শব্দটির অর্থ 'হট' এবং 'ডগ' শব্দগুলোর অন্য প্রসঙ্গে ব্যবহৃত অর্থ থেকে সম্পূর্ণ ভিন্ন। যদি আমরা 'হট' এবং 'ডগ' শব্দগুলোকে সবসময় একই ভেক্টর দিয়ে উপস্থাপন করি, তাহলে এটি আমাদের মডেলকে বিভ্রান্ত করতে পারে।

এই সমস্যার সমাধানে, **এন-গ্রাম উপস্থাপনাগুলো** প্রায়ই ডকুমেন্ট শ্রেণীবিন্যাস পদ্ধতিতে ব্যবহৃত হয়, যেখানে প্রতিটি শব্দ, দ্বি-শব্দ বা ত্রি-শব্দের ফ্রিকোয়েন্সি শ্রেণীবিন্যাসকারী প্রশিক্ষণের জন্য একটি কার্যকর বৈশিষ্ট্য। উদাহরণস্বরূপ, বাইগ্রাম উপস্থাপনায়, আমরা মূল শব্দগুলোর পাশাপাশি সমস্ত শব্দ জোড়াকে ভোকাবুলারিতে যোগ করব।

নিচে Scikit Learn ব্যবহার করে কীভাবে একটি বাইগ্রাম ব্যাগ অফ ওয়ার্ডস উপস্থাপনা তৈরি করা যায় তার একটি উদাহরণ দেওয়া হলো:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-gram পদ্ধতির প্রধান অসুবিধা হলো, শব্দভাণ্ডারের আকার অত্যন্ত দ্রুত বৃদ্ধি পেতে শুরু করে। বাস্তবে, আমাদের N-gram উপস্থাপনাকে কিছু মাত্রা হ্রাস করার কৌশলের সাথে সংযুক্ত করতে হয়, যেমন *embeddings*, যা আমরা পরবর্তী ইউনিটে আলোচনা করব।

আমাদের **AG News** ডেটাসেটে N-gram উপস্থাপন ব্যবহার করতে হলে, আমাদের একটি বিশেষ ngram শব্দভাণ্ডার তৈরি করতে হবে:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


আমরা উপরের কোডটি ব্যবহার করে ক্লাসিফায়ার প্রশিক্ষণ দিতে পারতাম, তবে এটি খুবই মেমোরি-অকার্যকর হতো। পরবর্তী ইউনিটে, আমরা এম্বেডিং ব্যবহার করে বিগ্রাম ক্লাসিফায়ার প্রশিক্ষণ দেব।

> **Note:** আপনি শুধুমাত্র সেই ngrams রাখতে পারেন যা নির্দিষ্ট সংখ্যার চেয়ে বেশি বার টেক্সটে উপস্থিত হয়। এটি নিশ্চিত করবে যে কম ঘনঘন বিগ্রামগুলো বাদ যাবে এবং ডাইমেনশনালিটি উল্লেখযোগ্যভাবে কমে যাবে। এটি করতে, `min_freq` প্যারামিটারকে একটি উচ্চতর মানে সেট করুন এবং ভোকাবুলারির দৈর্ঘ্যের পরিবর্তন পর্যবেক্ষণ করুন।


## টার্ম ফ্রিকোয়েন্সি ইনভার্স ডকুমেন্ট ফ্রিকোয়েন্সি (TF-IDF)

BoW উপস্থাপনায়, শব্দের উপস্থিতি সমানভাবে ওজন দেওয়া হয়, শব্দটি কী তা বিবেচনা না করেই। তবে, এটা স্পষ্ট যে সাধারণ শব্দ যেমন *a*, *in* ইত্যাদি শ্রেণীবিন্যাসের জন্য বিশেষায়িত শব্দগুলোর তুলনায় অনেক কম গুরুত্বপূর্ণ। প্রকৃতপক্ষে, বেশিরভাগ NLP কাজের ক্ষেত্রে কিছু শব্দ অন্যগুলোর তুলনায় বেশি প্রাসঙ্গিক।

**TF-IDF** এর পূর্ণরূপ হলো **টার্ম ফ্রিকোয়েন্সি–ইনভার্স ডকুমেন্ট ফ্রিকোয়েন্সি**। এটি ব্যাগ অফ ওয়ার্ডসের একটি পরিবর্তিত রূপ, যেখানে একটি শব্দের উপস্থিতি নির্দেশ করতে 0/1 বাইনারি মানের পরিবর্তে একটি ভাসমান-বিন্দু মান ব্যবহার করা হয়, যা কর্পাসে শব্দের উপস্থিতির ফ্রিকোয়েন্সির সাথে সম্পর্কিত।

আরও আনুষ্ঠানিকভাবে, একটি শব্দ $i$ এর ওজন $w_{ij}$ একটি ডকুমেন্ট $j$ এ সংজ্ঞায়িত করা হয়:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
যেখানে
* $tf_{ij}$ হলো $i$ শব্দটি $j$ ডকুমেন্টে যতবার উপস্থিত হয়েছে, অর্থাৎ আমরা আগে যে BoW মান দেখেছি
* $N$ হলো সংগ্রহে থাকা ডকুমেন্টের সংখ্যা
* $df_i$ হলো পুরো সংগ্রহে $i$ শব্দটি উপস্থিত থাকা ডকুমেন্টের সংখ্যা

TF-IDF মান $w_{ij}$ একটি ডকুমেন্টে শব্দটি যতবার উপস্থিত হয় তার সাথে আনুপাতিকভাবে বৃদ্ধি পায় এবং কর্পাসে শব্দটি উপস্থিত থাকা ডকুমেন্টের সংখ্যার দ্বারা সামঞ্জস্য করা হয়, যা এই বিষয়টি সমাধান করতে সাহায্য করে যে কিছু শব্দ অন্যগুলোর তুলনায় বেশি ঘন ঘন উপস্থিত হয়। উদাহরণস্বরূপ, যদি একটি শব্দ *প্রতিটি* ডকুমেন্টে উপস্থিত থাকে, $df_i=N$, এবং $w_{ij}=0$, এবং সেই শব্দগুলো সম্পূর্ণভাবে উপেক্ষা করা হবে।

আপনি সহজেই Scikit Learn ব্যবহার করে টেক্সটের TF-IDF ভেক্টরাইজেশন তৈরি করতে পারেন:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## উপসংহার

যদিও TF-IDF উপস্থাপনাগুলি বিভিন্ন শব্দের জন্য ফ্রিকোয়েন্সি ওজন প্রদান করে, সেগুলি অর্থ বা ক্রম উপস্থাপন করতে অক্ষম। ১৯৩৫ সালে বিখ্যাত ভাষাবিদ জে. আর. ফার্থ বলেছিলেন, "একটি শব্দের সম্পূর্ণ অর্থ সর্বদা প্রাসঙ্গিক, এবং প্রসঙ্গ ছাড়া অর্থের কোনো অধ্যয়নকে গুরুত্ব সহকারে নেওয়া যায় না।" আমরা এই কোর্সে পরে শিখব কীভাবে ভাষা মডেলিং ব্যবহার করে পাঠ্য থেকে প্রাসঙ্গিক তথ্য ধারণ করা যায়।



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা তার জন্য দায়ী থাকব না।
